
1. <font color=royalblue>**Workbook defines a trinomial function which can be used  to evaluate Vanilla and Single Barrier Option**</font


2. <font color=royalblue>**Supports both american and european excercise style**</font>


3. <font color=royalblue>**Supported barrier pricing:**</font>
    1. **UP-OUT**
    2. **UP-IN**
    3. **DOWN-OUT**
    4. **DOWN-IN**


4.<font color=royalblue>**Note:Does not yet support dividend payments and rebates. Will be incoporated in future versions**</font>


5.<font color=royalblue>**The function has 11 inputs:**</font>
-  ***S0 -> Initial Price***
-  ***strike -> strike price***
-  ***sigma -> underlying volatility***
-  ***T -> time to expiry in years***
-  ***N -> number of steps for the tree***
-  ***r -> interest rate***
-  ***OptionFlag -> 'C' (Call)/ 'P' (Put)***
-  ***ExcerciseType -> 'A' (American)/ 'P' (European)***
- ***Barrier -> 'Y' (Yes)/ 'N' (No)***
-  ***BarrierType -> 'UP-OUT'/'UP-IN'/'DOWN-OUT'/'DOWN-IN'***  (**Note: Only required if barrier flag is 'Y'**)
-  ***H -> Barrier Price*** (**Note: Only required if barrier flag is 'Y'**)


In [1]:
import numpy as np
import math


In [2]:
np.set_printoptions(precision=3,suppress=True)


In [3]:
#Function Starts here:
def trinomialtreepricing(S0,strike,sigma,T,N,r,OptionFlag,ExcerciseType,Barrier,BarrierType,H):
    t= T/N
    
    #Size of UpMove (u) and DownMove (d)
    u = math.exp(sigma*math.sqrt(2*t))
    d = 1/u
    
    #Discount Factor
    df = math.exp(-r*t)
    
    #Probability of UpMove(p_u),DownMove (p_d) & sideways move (p_m)
    component_1 = math.exp(r*t/2)
    component_2 = math.exp(-sigma*math.sqrt(t/2))
    
    denominator_1 = math.exp(sigma*math.sqrt(t/2))
    denominator_2 = math.exp(-sigma*math.sqrt(t/2))
    
    p_u = math.pow((component_1 - component_2)/(denominator_1 - denominator_2),2)
    p_d = math.pow((denominator_1 - component_1)/(denominator_1 - denominator_2),2)
    p_m = 1- p_u - p_d
    
    #empty array to create stock price tree
    S = np.zeros((N*2+1,N+1))
    midpoint = int((N*2+1)/2)
    
    S[midpoint] = S0
    
    #For Loop to populate the price tree S in two tranches
    #The upper tranche or half of the pricing tree
    for i in range(midpoint-1,-1,-1):
        for j in range(1,N+1):
            S[i,j] = S[i+1,j-1]*u
    
    #The lower tranche or half of the pricing tree
    for i in range(midpoint+1,N*2+1):
        for j in range(1,N+1):
            S[i,j] = S[i-1,j-1]*d

    
    ##empty array to create option payoff
    P = np.zeros((N*2+1,N+1))
    
    #For Loop to fill the option payoff array P
    #for i in range(0,S.shape[0]):
    for j in range(N,-1,-1):
        #for j in range(0,S.shape[1]):
        for i in range(len(P) - (j+N)-1,j+N+1):
            #print(i,j)
            if Barrier == 'Y':
                if (BarrierType == 'UP-OUT' and S[i,j] >= H):
                    P[i,j] == 0   
                elif (BarrierType == 'UP-IN' and S[i,j] < H):
                        P[i,j] == 0
                elif (BarrierType == 'DOWN-OUT' and S[i,j] <= H):
                    P[i,j] == 0
                elif (BarrierType == 'DOWN-IN' and S[i,j] > H):
                    P[i,j] == 0 
                elif OptionFlag == 'C':
                    P[i,j] = max(S[i,j] - strike,0)
                elif OptionFlag == 'P':
                    P[i,j] = max(strike - S[i,j] ,0)
                else:
                    0
            elif Barrier == 'N':
                if OptionFlag == 'C':
                    P[i,j] = max(S[i,j] - strike,0)
                
                elif OptionFlag == 'P':
                    P[i,j] = max(strike - S[i,j] ,0)
                else:
                    0
            
    #empty array of option price
    option_price = np.zeros((N*2+1,N+1))
    
    #Discounted recursive option price base on Barrier Condition and whether excercise type is American or European
    for j in range(N,-1,-1):
        if  j == N:
            option_price[:,j] = P[:,j] 
        else:
            for i in range(len(option_price) - (j+N)-1,j+N+1):
                if Barrier == 'Y':
                    if (BarrierType == 'UP-OUT' and S[i,j] >= H):
                        0
                    elif (BarrierType == 'UP-IN' and S[i,j] < H):
                        0
                    elif (BarrierType == 'DOWN-OUT' and S[i,j] <= H):
                        0
                    elif (BarrierType == 'DOWN-IN' and S[i,j] >= H):
                        0   
                    else:
                        if ExcerciseType == 'A': 
                            option_price[i,j] = max((option_price[i,j+1]*p_m + option_price[i-1,j+1]*p_u + option_price[i+1,j+1]*p_d)*df,P[i,j])
                        elif ExcerciseType == 'E':
                            option_price[i,j] = (option_price[i,j+1]*p_m + option_price[i-1,j+1]*p_u + option_price[i+1,j+1]*p_d)*df
                elif Barrier == 'N':
                     if ExcerciseType == 'A': 
                            option_price[i,j] = max((option_price[i,j+1]*p_m + option_price[i-1,j+1]*p_u + option_price[i+1,j+1]*p_d)*df,P[i,j])
                     elif ExcerciseType == 'E':
                        option_price[i,j] = (option_price[i,j+1]*p_m + option_price[i-1,j+1]*p_u + option_price[i+1,j+1]*p_d)*df

    #Just fuff to prettify the print output and return the option price, probabilities and size of up and downmove
    if N > 9:
        print('\033[1m'+ 'For visibility nodes restricted to first 9 time steps'+ "\n" \
              '\033[94m' + '\033[1m' + 'Price Tree'+ "\n" \
              + '\033[1;36m'  + '\033[33m' + str(S[N-9+1:N+9,:9]))
        
        print('\033[1m'+ 'For visibility nodes restricted to first 9 time steps'+ "\n" \
              + '\033[33m' + 'Option Price Tree'+ "\n" \
              + '\033[0m' + str(option_price[N-9+1:N+9,:9]))

    else:
        print('\033[94m' + '\033[1m' + 'Price Tree'+ "\n" + '\033[0m' + str(S))
        print('\033[1;36m'  + '\033[33m' + 'Option Price Tree'+ "\n" + '\033[0m' + str(option_price))

    return('Option Price: ' + str(np.round(option_price[midpoint,0],3)),\
           'P(U): ' + str(np.round(p_u,3)),'P(M): ' + str(np.round(p_m,3)),'P(D): ' + str(np.round(p_d,3)),\
          'Upmove: ' + str(np.round(u,3)),'Downmove: ' + str(np.round(d,3)))

###Function Ends here

<font color=royalblue>**User Input Section**</font>


In [4]:
S0 = 100
sigma = 0.40
T = 1
N = 5
strike = 120
r = 0.05
OptionFlag = 'P'
ExcerciseType = 'E'
Barrier = 'N'

#Only required if it Barrier flag is 'Y'
BarrierType = None
H = None



<font color=royalblue>**The function produces 3 outputs:**</font>
1. Price Tree of the underlying 
2. Option Price Tree using backward induction
3. List of:
    1. Option,P(U) - Probability of Upmove, 
    2. P(M) - Probability of horizontal move, 
    3. P(D) - Probability of down move , 
    4. Upmove, 
    5. Downmove

In [5]:
trinomialtreepricing(S0,strike,sigma,T,N,r,OptionFlag,ExcerciseType,Barrier,BarrierType,H)

Price Tree
[[  0.      0.      0.      0.      0.    354.278]
 [  0.      0.      0.      0.    275.09  275.09 ]
 [  0.      0.      0.    213.603 213.603 213.603]
 [  0.      0.    165.858 165.858 165.858 165.858]
 [  0.    128.786 128.786 128.786 128.786 128.786]
 [100.    100.    100.    100.    100.    100.   ]
 [  0.     77.648  77.648  77.648  77.648  77.648]
 [  0.      0.     60.292  60.292  60.292  60.292]
 [  0.      0.      0.     46.816  46.816  46.816]
 [  0.      0.      0.      0.     36.352  36.352]
 [  0.      0.      0.      0.      0.     28.226]]
Option Price Tree
[[ 0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     2.736  1.345  0.     0.   ]
 [ 0.    11.671 10.031  7.981  5.187  0.   ]
 [25.162 24.34  23.375 22.228 20.879 20.   ]
 [ 0.    39.967 40.134 40.465 41.158 42.352]
 [ 0.     0.    56.276 57.331 58.514 59.708]
 [ 0.     0.     0.    70.808 71.99  73.184]
 [ 0

('Option Price: 25.162',
 'P(U): 0.238',
 'P(M): 0.5',
 'P(D): 0.262',
 'Upmove: 1.288',
 'Downmove: 0.776')